# Uge11:  PDF downloader

### Opgave beskrivelse


Din kunde har et gammelt Python script, som efter et par år i drift ikke fungerer mere.

Du modtager følgende beskrivelse fra din kunde:

 

”Kære Konsulent 

Som aftalt har du hermed listen (inkl. metadata) ang. de rapporter vi gerne vil have downloaded (”GRI_2017_2020”).,

,

,

,

Din opgave er nu i dag, at:

-  Du skal beslutte dig for om du vil opdatere det skrevne Python script eller lavet et nyt. Du har frit valg mht. programmeringssprog, da kunden netop efterspørger din hjælp.

- En featureliste/Kravspec skal udarbejdes inden du går i gang med din løsning.

## Introduktion:

Formålet med denne automatiserings PDF-Downloader opgave er at processen med at downloade PDF-rapporter fra en liste over 

webadresser given i Excel-filer **GRI_2017_2020** og **Metadata2006_2016**.

Ved at udnytte Pythons muligheder og biblioteker som pandas og urllib sigter vi mod at udvikle en robust og effektiv løsning,

der eliminerer behovet for manuel indgriben.

I denne dokumentationen vil vi dykke ned i Python-kodeimplementeringen og demonstrere funktionaliteten gennem en interaktiv

terminal. Ved at automatisere PDF-rapportdownloads sparer vi ikke kun tid og kræfter, men sikrer også nøjagtighed og konsistens i datahentning.


## Kravspecifikationer:

- Script skal være pålidelig og i stand til at håndtere forskellige scenarier, herunder korrekte og forkerte URL,er samt eventuelle fejl under downloadprocessen.


- Script kan håndtere store mængder data hurtigt og er effektiv.

- Script skal være i stand til at håndtere fejl og uventede situationer på en elegent måde, således den ikke bryder ned eller stopper midt i processen.

- Script skal tiilpasses forskellige dataformater.

- Script skal håndtere og  download fra usikre kilder, implementere passende sikkerhedsforanstaltninger for at beskytte mod trusler og sikre datasikkerheden.


Lad os starte med importring vigtige biblioteker.

In [7]:
import pandas as pd  #import neccessary libraries
from urllib import request, error

## Define Funtions:

Funktionen formål er at downloade en PDF-rapport fra en given URL og gemme den lokalt på computeren.


Parametre:

**url:** URL'en for PDF-rapporten, som skal downloades.

**save_path:** Filstien, hvor den downloadede rapport skal gemmes.

**Try-Except-blok:**

Koden er omgivet af en try-except-blok for at håndtere eventuelle fejl under downloadprocessen.

Hvis der opstår en URLError (f.eks. forbindelsesproblemer), eller en anden type fejl, vil funktionen returnere False og udskrive en fejlbesked.


**Funktionen** åbner en forbindelse til den angivne URL og kontrollerer, om svaret har status 200 (OK).

Hvis svaret er 200, åbner den den angivne filsti i binært skrivetilstand ('wb') og skriver indholdet af svaret til filen.
Hvis svaret ikke er 200, returnerer den False for at angive, at downloaden mislykkedes.

In [2]:
# Function to download PDF reports
def download_report(url, save_path):
    try:
        with request.urlopen(url) as response:
            if response.status == 200:
                with open(save_path, 'wb') as file:
                    file.write(response.read())
                return True
            else:
                return False
    except error.URLError as e:
        print(f"Error downloading report: {e.reason}")
        return False
    except Exception as e:
        print(f"Error downloading report: {e}")
        return False

## Læsning af data-filer til pandas DataFrames: ( nedenudner-kode)


Indlæsning til to Excel-filer, **'Metadata2006_2016.xlsx'** og **'GRI_2017_2020.xlsx'**, er angivet i variablerne **metadata_file** og **report_list_file**.

Tomme værdier i 'Pdf_URL'-kolonnen i **report_list_df** erstattes med en tom streng (''), så der ikke opstår problemer senere under behandlingen.

En ny kolonne med navnet **'Report Name'** tilføjes til metadata_df, og data fra **'BRnum'-kolonnen** i **report_list_df**

kopieres ind i denne nye kolonne. Dette antages at være en måde at knytte rapportnavne til metadataen fra den anden fil.

Denne kode sikrer, at data fra de angivne Excel-filer indlæses korrekt i DataFrames, og eventuelle mangler eller nødvendige tilføjelser behandles, så dataene er klar til yderligere behandling eller analyse.


In [8]:
# Read Excel files
metadata_file = 'Metadata2006_2016.xlsx'
report_list_file = 'GRI_2017_2020.xlsx'

metadata_df = pd.read_excel(metadata_file, engine='openpyxl')
report_list_df = pd.read_excel(report_list_file, engine='openpyxl')

# Fix empty values in Pdf_URL column
report_list_df['Pdf_URL'].fillna('', inplace=True)

# Add 'Report Name' column to metadata_df
metadata_df['Report Name'] = report_list_df['BRnum']


## Function to download reports by index:

Funktionen **download_report_by_index** tager et indeks som inputparameter og downloader rapporten med det angivne indeks fra **report_list_df**.


Først kontrolleres, om det angivne indeks er gyldigt (dvs. ikke negativt og inden for rækkevidden af rækker i **report_list_df**).

Herefter hentes dataene for den pågældende række (rapport) fra **report_list_df**.

URL'en til PDF-rapporten og rapportens BR-nummer (antaget som et identifikationsnummer) udtrækkes fra den pågældende række.


Der kontrolleres, om PDF-URL'en ikke er tom. Hvis den ikke er det, forsøges rapporten downloadet ved hjælp af **download_report**-funktionen.


Afhængigt af downloadstatus opdateres **'Status'-kolonnen** i status_df med enten **'Downloaded'** eller **'Not downloaded'**.


Der udskrives en meddelelse om, hvorvidt rapporten blev downloadet eller ej, og processen fortsætter til den næste rapport.


Denne kode sikrer, at downloadstatussen for hver rapport registreres i **status_df** efterhånden som rapporterne downloades, hvilket giver brugeren en oversigt over, hvilke rapporter der er blevet hentet succesfuldt.


In [4]:
# Create a new DataFrame with 'Report Name' and 'Status' columns
status_df = pd.DataFrame({'Report Name': metadata_df['Report Name'], 'Status': ''})

# Function to download report by index
def download_report_by_index(index):
    if index < 0 or index >= len(report_list_df):
        print("Invalid index.")
        return
    row = report_list_df.iloc[index]
    pdf_url = row['Pdf_URL']
    br_nummer = row['BRnum']
    
    # Check if Pdf_URL is not empty
    if pdf_url:
        success = download_report(pdf_url, f"{br_nummer}.pdf")
        if success:
            print(f"Report {br_nummer} was downloaded.")
            status_df.at[index, 'Status'] = 'Downloaded'
        else:
            print(f"Report {br_nummer} could not be downloaded.")
            status_df.at[index, 'Status'] = 'Not downloaded'
    else:
        print(f"No PDF URL found for index {index}. Skipping.")


## Test og Implement an interactive terminal:

For User input laver vi intraktiv terminal, hvor brugeren kan angive indekset for den rapport, der skal downloades.

**Download af rapport**

Hvis brugeren angiver et gyldigt indeks, kaldes funktionen **download_report_by_index** for at downloade den tilsvarende rapport.

Før downloaden aktiveres, justeres indekset ved at trække 1 fra. Dette skyldes, at brugerinput indekset baseres på 1, mens Python-indeksering starter fra 0.

Hvis brugeren indtaster exit_index, i.e -1 vil while-løkken blive afbrudt, og interaktionen afsluttes.
python

koden skriver også statusen for rapporterne til en Excel-fil ved hjælp af **to_excel()**-funktionen fra pandas.

Til sidst udskrives en meddelelse om, at statussen for rapporterne er gemt i filen **'Report_Status.xlsx'**.

In [5]:
# Interactive terminal
exit_index = -1  # Define the exit index
while True:
    try:
        index = int(input(f"Enter the index of the report to download ({exit_index} to exit): "))
        if index == exit_index:
            break
        if index < 0:
            print("Invalid index. Index must be non-negative.")
            continue
        download_report_by_index(index - 1)  # Adjusted index by subtracting 1
    except ValueError:
        print("Invalid input. Please enter a valid index.")
       
    
# Write status DataFrame to a new Excel file
status_file = 'Report_Status.xlsx'
status_df.to_excel(status_file, index=False)
print(f"Status of reports saved to '{status_file}'.")

Enter the index of the report to download (-1 to exit): 0
Invalid index.
Enter the index of the report to download (-1 to exit): 1
Report BR50041 was downloaded.
Enter the index of the report to download (-1 to exit): 2
Report BR50042 was downloaded.
Enter the index of the report to download (-1 to exit): 3
Error downloading report: Forbidden
Report BR50043 could not be downloaded.
Enter the index of the report to download (-1 to exit): 4
Error downloading report: [WinError 3] Den angivne sti blev ikke fundet: 'C:\\Users\\miller\\Downloads\\AA_AR2017.pdf'
Report BR50044 could not be downloaded.
Enter the index of the report to download (-1 to exit): 5
Report BR50045 was downloaded.
Enter the index of the report to download (-1 to exit): 6
No PDF URL found for index 5. Skipping.
Enter the index of the report to download (-1 to exit): 1000
Error downloading report: Not Found
Report BR51040 could not be downloaded.
Enter the index of the report to download (-1 to exit): 2000
Report BR5204


## End of Notebook

This notebook is created by: Shakeel Ahmed.